# Testing the equilibrate code, and notes

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import scipy as sci
import sys

In [ ]:
from thermoengine import core, phases, model, equilibrate

In [ ]:
np.set_printoptions(linewidth=200, precision=1)

### T,P

In [ ]:
t = 1050.0
p = 1750.0

## Flags
By default, both of these flags are False.
- lagrange_use_omni forces the system to use ONLY the omnicomponent phase to balance the imposed chemical potential constraint
- lagrange_no_mol_deriv forces the construction of the Khorzhinskii potential to use a constant value for the imposed potential, equal to that of the imposed constraint

In [ ]:
lagrange_use_omni = True
lagrange_no_mol_deriv = True

## Create phases for equilibrium assemblages

In [ ]:
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Corundum = modelDB.get_phase('Crn')

This is the starting composition of the system (moles of components, liquid first, then feldspar)

In [ ]:
if lagrange_use_omni:
    nref = np.array([1.11066366e+00, 1.00126660e-03, 2.34951267e-01, 1.29624365e-03, 0.00000000e+00, 3.29174461e-03, 
                     0.00000000e+00, 3.72167803e-04, 0.00000000e+00, 0.00000000e+00, 7.66351040e-03, 6.42125223e-02,
                     1.03614268e-01, 0.00000000e+00, 3.05297749e-01, 5.67713421e-06, 4.18932529e-06, 1.33540497e-07])
else:
    nref = np.array([1.11066104e+00, 1.00126660e-03, 2.34939818e-01, 1.29624365e-03, 0.00000000e+00, 3.29174461e-03,
                     0.00000000e+00, 3.72167803e-04, 0.00000000e+00, 0.00000000e+00, 7.66655039e-03, 6.42116348e-02,
                     1.03613656e-01, 0.00000000e+00, 3.05297749e-01, 7.45226819e-06, 1.14933970e-06, 7.45751928e-07])
nref_l = nref[:-3]
nref_f = nref[-3:]
nref_l,nref_f

Phase properties - liquid

In [ ]:
gLiq = Liquid.gibbs_energy(t,p,mol=nref_l,deriv={"dmol":0})
dgLiq = Liquid.gibbs_energy(t,p,mol=nref_l,deriv={"dmol":1})[0]
d2gLiq = Liquid.gibbs_energy(t,p,mol=nref_l,deriv={"dmol":2})[0]
d3gLiq = Liquid.gibbs_energy(t,p,mol=nref_l,deriv={"dmol":3})[0]

Phase properties - feldspar

In [ ]:
gFld = Feldspar.gibbs_energy(t,p,mol=nref_f,deriv={"dmol":0})
dgFld = Feldspar.gibbs_energy(t,p,mol=nref_f,deriv={"dmol":1})[0]
d2gFld = Feldspar.gibbs_energy(t,p,mol=nref_f,deriv={"dmol":2})[0]
d3gFld = Feldspar.gibbs_energy(t,p,mol=nref_f,deriv={"dmol":3})[0]

# Algorithm Testing
Output from notebook 7b  
Rhyolite liquid, MELTS 1.0.2 model  
Supersaturation with feldspar  
Forced to be at corundum saturation

Khorzhinskii potential:  
$L = G\left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\Phi \left( {{\bf{n}},T,P} \right)$  

where $\Phi \left( {{\bf{n}},T,P} \right) = {{\bf{r}}^T}\frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$ is a general chemical potential constraint and $\bf{r}$ is a vector of stoichiometric reaction coefficients relating system phase components to the imposed potential.   

The first derivative (gradient) is:  
$\frac{{\partial L}}{{\partial {\bf{n}}}} = \frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - \left( {{{\bf{r}}^T}\frac{{\partial {\bf{n}}}}{{\partial {\bf{n}}}} + {\bf{n}}\frac{{\partial {{\bf{r}}^T}}}{{\partial {\bf{n}}}}} \right)\Phi \left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  

note $\frac{{\partial {\bf{n}}}}{{\partial {\bf{n}}}} = {\bf{I}}$, $\frac{{\partial {{\bf{r}}^T}}}{{\partial {\bf{n}}}} = {\bf{0}}$, ${{\bf{r}}^T}{\bf{I}} = {\bf{r}}$  

$\frac{{\partial L}}{{\partial {\bf{n}}}} = \frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - {\bf{r}}\Phi \left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  

The second derivative (hessian) is:  
$\frac{{{\partial ^2}L}}{{\partial {{\bf{n}}^2}}} = \frac{{{\partial ^2}G\left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}} - \frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}{{\bf{r}}^T} - {\bf{r}}\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{{\partial ^2}\Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}$  

The Lagrangian is:  
$\Lambda  = L - \lambda \left( {\Phi \left( {{\bf{n}},T,P} \right) - {\Phi ^{fix}}\left( {T,P} \right)} \right)$  

and its contribution to the second derivative is:  
$\frac{{{\partial ^2}\Lambda }}{{\partial {{\bf{n}}^2}}} =  - \lambda \frac{{{\partial ^2}\Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}$  

Note that as $\Phi \left( {{\bf{n}},T,P} \right) = {{\bf{r}}^T}\frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$, the derivative is:  
$\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} = \frac{{{\partial ^2}G\left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}{\bf{r}}$

### Constraint matrices (columns elements)
_f Feldspar  
_q Quartz  
_c Corundum  
_qc Quartz+Corundum  
_qf Quartz+Feldspar  
_ox Oxygen

We only use the corundum constraint matrix in the following

In [ ]:
CTf_f = np.array([[0., 8., 1., 0., 1., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                  [0., 8., 0., 0., 2., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
                  [0., 8., 0., 0., 1., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
CTf_q = np.array([[0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
CTf_c = np.array([[0., 3., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
CTf_qc = np.array([[0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 3., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
CTf_qf = np.array([[0., 8., 1., 0., 1., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 8., 0., 0., 2., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
                   [0., 8., 0., 0., 1., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
CTf_ox = np.array([[0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
#CTf_m = np.array([[0.4388],[0.0104], [0.5508]]) # Feldspar composition at supersaturation
#CTf = CTf_m*CTf
CTf = CTf_c
CTf

Inflated constraint matrix

In [ ]:
CTf = np.pad(CTf,((0,CTf.shape[1]-CTf.shape[0]),(0,0)),mode='constant')
print (CTf.shape)
CTf

Utility function to eliminate rounding "zeros"

In [ ]:
filtr = lambda x : x if abs(x) > float(1000*sys.float_info.epsilon) else 0
vfiltr = np.vectorize(filtr, otypes=[float])

Stoichiometric constraint matrices:  
- A (silicate liquid, MELTS model, columns = components, rows = elements)
- A_w like A, additional column for water (not used in the following)
- A_f like A, additional three columns for feldspar 

In [ ]:
A = np.array([[0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2. ],
              [2.,  2.,  3.,  3.,  4.,  4.,  2.,  4.,  2.,  2.,  3.,  3.,  4.,  8.,  1. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0. ],
              [1.,  0.,  0.,  0.,  0.,  1.,  0.5, 1.,  0.5, 0.5, 1.,  1.,  1.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  3.,  0. ],
              [0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0. ]])
A_w = np.array([[0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., 2. ],
              [2.,  2.,  3.,  3.,  4.,  4.,  2.,  4.,  2.,  2.,  3.,  3.,  4.,  8.,  1., 1. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 0. ],
              [1.,  0.,  0.,  0.,  0.,  1.,  0.5, 1.,  0.5, 0.5, 1.,  1.,  1.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  3.,  0., 0. ],
              [0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., 0. ]])
A_f = np.array([[0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., 0., 0., 0. ],
              [2.,  2.,  3.,  3.,  4.,  4.,  2.,  4.,  2.,  2.,  3.,  3.,  4.,  8.,  1., 8., 8., 8. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0., 1., 0., 0. ],
              [0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 1., 2., 1. ],
              [1.,  0.,  0.,  0.,  0.,  1.,  0.5, 1.,  0.5, 0.5, 1.,  1.,  1.,  0.,  0., 3., 2., 3. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 0., 0., 1. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  3.,  0., 0., 1., 0. ],
              [0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ],
              [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0., 0., 0., 0. ]])

We are going to convert the A_f matrix from an element basis to the basis of the omnicomponent phase (liquid).  
First, invert the A matrix (columns = liquid components, rows = elements) ...

In [ ]:
Ainv = np.linalg.inv(A)

Second, project the A_f matrix by multiplying it by this inverse.  
This process yields Acomp, a bulk composition constraint matrix with columns = liquid components, rows = liquid components  
Note, that this process partitions the constraint matrix into an identity matrix and a reaction matrix whose columns describe the stoichiomtry of reactions between feldspar components and liquid components. 

In [ ]:
Acomp = np.matmul(Ainv, A_f)
Acomp

Row of constraints to be added to a reduced A_f matrix for the chemical potential of alumina:  

Constraints:  
$\Phi \left( {{\bf{n}},T,P} \right) = {{\bf{r}}^T}\frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  

Constraint derivatives:   
$\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} = \frac{{{\partial ^2}G\left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}{\bf{r}}$  


Balanced reaction involving silicate liquid + feldspar constraining the chemical potential of alumina

In [ ]:
if lagrange_use_omni:
    Areact = np.hstack((Acomp[:,:15],np.zeros((15,3))))
else:
    Areact = Acomp
react, res, rank, s = np.linalg.lstsq(Areact, np.matmul(Ainv,CTf_c.T), rcond=None)
reaction = react.T
reaction

Computed chemical potential of the constraining reaction:

In [ ]:
mu_est = np.matmul(reaction[0], np.hstack((dgLiq,dgFld)))
mu_est - Corundum.gibbs_energy(t,p)

In [ ]:
Con = np.matmul(reaction, sci.linalg.block_diag(*[d2gLiq,d2gFld]))
Con

Next, project the constraint matrix (columns = elements, rows = imposed chemical potential constraints) by multiplying by Ainv, and then construct the null space of this projection.  
The null space shows which liquid component and feldspar component mole numbers are constrained.

In [ ]:
ns = sci.linalg.null_space(np.matmul(Ainv,CTf.T).T)
vfiltr(ns)

Finally, form the final constraint matrix by  
- First, projecting the Acomp matrix into the null space of the imposed chemical potential constraints, and
- Second, stacking that projection on top of the chemical potential derivative coinstraint vector, Con

In [ ]:
A = np.vstack((vfiltr(np.matmul(ns.T,Acomp)), Con))
A

Decompose the A matrix into an orthogonal projection operator for the gradient and hessian

In [ ]:
row,col = A.shape
df = col - row
R, Q = sci.linalg.rq(A, mode='full')
R11 = vfiltr(R[:,df:])
Q1 = vfiltr(Q[df:,:])
Q2 = vfiltr(Q[0:df,:])
Q2

The Korzhinskii potential:  
$L = G\left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\Phi \left( {{\bf{n}},T,P} \right)$  

has the first derivative with respect to composition (n contains liquid + feldspar component mole numbers):  
$\frac{{\partial L}}{{\partial {\bf{n}}}} = \frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - \left( {{{\bf{r}}^T}\frac{{\partial {\bf{n}}}}{{\partial {\bf{n}}}} + {\bf{n}}\frac{{\partial {{\bf{r}}^T}}}{{\partial {\bf{n}}}}} \right)\Phi \left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  

and from the identities:
$\frac{{\partial {\bf{n}}}}{{\partial {\bf{n}}}} = {\bf{I}}$ and $\frac{{\partial {{\bf{r}}^T}}}{{\partial {\bf{n}}}} = {\bf{0}}$, ${{\bf{r}}^T}{\bf{I}} = {\bf{r}}$  

The expression for the gradient is:  
$\frac{{\partial L}}{{\partial {\bf{n}}}} = \frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - {\bf{r}}\Phi \left( {{\bf{n}},T,P} \right) - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  

There are three terms:
- the Gibbs energy gradient
- the chemical potential part (second term rhs above, _add_1 below)
- the chemical potential derivative part (third term rhs above, _add_2 below)

In [ ]:
g = np.hstack((dgLiq,dgFld))
g = np.reshape(g,(g.size,1))
if lagrange_no_mol_deriv:
    g_add_1 = -Corundum.gibbs_energy(t,p)*reaction
else:
    g_add_1 = -np.matmul(reaction,g)[0][0]*reaction
g_add_1 = np.reshape(g_add_1,(g_add_1.size,1))
moles = np.matmul(reaction, nref)[0]
if lagrange_no_mol_deriv:
    g_add_2 = np.zeros(g.shape)
else:
    g_add_2 = -moles*np.matmul(reaction, sci.linalg.block_diag(*[d2gLiq,d2gFld]))
    g_add_2 = np.reshape(g_add_2,(g_add_2.size,1))

The hessian of the Korzhinskii potential is:  
$\frac{{{\partial ^2}L}}{{\partial {{\bf{n}}^2}}} = \frac{{{\partial ^2}G\left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}} - \frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}{{\bf{r}}^T} - {\bf{r}}\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} - \left( {{{\bf{r}}^T}{\bf{n}}} \right)\frac{{{\partial ^2}\Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}$  

There are four terms:
- the hessian of the Gibbs free energy
- a chemical potential derivative term (second term rhs above, _add_1 below)
- another chemical potential derivative term (third term rhs above, _add_2 below)
- the second chemical potential derivative term (fourth term rhs above, _add_3 below)

In [ ]:
H = sci.linalg.block_diag(*[d2gLiq,d2gFld])
if lagrange_no_mol_deriv:
    H_add_1 = np.zeros(H.shape)
    H_add_2 = np.zeros(H.shape)
else:
    H_add_1 = -np.outer(reaction,np.matmul(reaction, sci.linalg.block_diag(*[d2gLiq,d2gFld])))
    H_add_2 = -np.outer(np.matmul(reaction, sci.linalg.block_diag(*[d2gLiq,d2gFld])),reaction.T)
H_add_3_L = np.zeros(d2gLiq.shape)
H_add_3_F = np.zeros(d2gFld.shape)
def d3gP (index, d3g):
    nc = d3g.shape[0]
    result = np.zeros((nc,nc))
    for i in range(0,nc):
        for j in range(i,nc):
            for k in range(j,nc):
                if i == index:
                    result[j][k] = d3g[i][j][k]
                    result[k][j] = d3g[i][j][k]
                elif j == index:
                    result[i][k] = d3g[i][j][k]
                    result[k][i] = d3g[i][j][k]
                elif k == index:
                    result[i][j] = d3g[i][j][k]
                    result[j][i] = d3g[i][j][k]
    return result
for index,coeff in enumerate(reaction[0]):
    if index < 15:
        H_add_3_L += coeff*d3gP(index, d3gLiq)
    else:
        H_add_3_F += coeff*d3gP(index-15, d3gFld)
if lagrange_no_mol_deriv:
    H_add_3 = np.zeros(H.shape)
else:
    H_add_3 = -moles*sci.linalg.block_diag(*[H_add_3_L,H_add_3_F])

Lagrange multipliers constructed from the gradient and equility constraint matrix

In [ ]:
lagrange_m, res, rank, s = np.linalg.lstsq(A.T, g+g_add_1+g_add_2, rcond=None)
lagrange_m

Augmentation to the hessian matrix to transform it into the second derivative matrix of the Lagrangian.  

The Lagrangian:  
$\Lambda  = L - \lambda \left( {\Phi \left( {{\bf{n}},T,P} \right) - {\Phi ^{fix}}\left( {T,P} \right)} \right)$  
$\frac{{{\partial ^2}\Lambda }}{{\partial {{\bf{n}}^2}}} =  - \lambda \frac{{{\partial ^2}\Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}$  

Constraint derivatives:  
$\Phi \left( {{\bf{n}},T,P} \right) = {{\bf{r}}^T}\frac{{\partial G\left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}}$  
$\frac{{\partial \Phi \left( {{\bf{n}},T,P} \right)}}{{\partial {\bf{n}}}} = \frac{{{\partial ^2}G\left( {{\bf{n}},T,P} \right)}}{{\partial {{\bf{n}}^2}}}{\bf{r}}$

In [ ]:
lagrange_m[-1][0]
H_add_c = -lagrange_m[-1][0]*sci.linalg.block_diag(*[H_add_3_L,H_add_3_F])

In [ ]:
np.set_printoptions(linewidth=200, precision=8)

Project the gradient of the Korzhinskii function into the constraint null space 

In [ ]:
g_p = np.matmul(Q2, g+g_add_1+g_add_2)
g_p, g_p.shape

Project the hessian of the Lagrangian function into the constraint null space

In [ ]:
H_p = np.matmul(np.matmul(Q2, H+H_add_1+H_add_2+H_add_3+H_add_c), Q2.T)
H_p, H_p.shape

Solve the "quadratic search" sub-problem

In [ ]:
result, residuals, rank, S = np.linalg.lstsq(H_p, -g_p, rcond=None)
result, residuals, rank, S

Reconstruct the "quadratic search" direction vector

In [ ]:
n2 = np.matmul(Q2.T, result)
n2

For a unit step along the computed "quadratic search" direction, how do the mole numbers of phase components change?  

All mole numbers should be positive for a steplength of ~1, as we are adding feldspar to a liquid assemblage that is known to be saturated in feldspar.

In [ ]:
step = 1.0
loop = True
while loop:
    x = nref + n2[:,0]*step
    found = False
    for entry in [x < 0][0]:
        if entry:
            step /= 2.0
            found = True
    loop = found
print ("step length:", step)
for i in range(0,x.size):
    print ("{0:13.6e} {1:13.6e} {2:13.6e}".format(nref[i], x[i], x[i]-nref[i]))